In [1]:
import pandas as pd
import re

In [2]:
tweet = pd.read_csv('ressources/covid19_tweets.csv',parse_dates=['user_created','date'])

In [3]:
tweet

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,AJIMATI AbdulRahman O.,"Ilorin, Nigeria",Animal Scientist|| Muslim|| Real Madrid/Chelsea,2013-12-30 18:59:19,412,1609,1062,False,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,['WearAMask'],Twitter for Android,False
179104,Jason,Ontario,When your cat has more baking soda than Ninja ...,2011-12-21 04:41:30,150,182,7295,False,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,['COVID19'],Twitter for Android,False
179105,BEEHEMOTH ⏳,🇨🇦 Canada,⚒️ The Architects of Free Trade ⚒️ Really Did ...,2016-07-13 17:21:59,1623,2160,98000,False,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,NaN,Twitter Web App,False
179106,Gary DelPonte,New York City,"Global UX UI Visual Designer. StoryTeller, Mus...",2009-10-27 17:43:13,1338,1111,0,False,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",['COVID19'],Twitter for iPhone,False


## Hashtags

In [4]:
hastags = tweet.hashtags.dropna().apply(eval).explode().str.lower()

In [5]:
hastags[hastags.str.contains('covid')] = 'Covid, Covid19, Covid_19 ...'

In [6]:
hastags[hastags.str.contains('corona')] = 'coronavirus, corona ...'

In [7]:
hastags_values = hastags.value_counts().reset_index()
hastags_values

,hashtags,count
0,"Covid, Covid19, Covid_19 ...",107428
1,"coronavirus, corona ...",16420
2,pandemic,1625
3,india,1193
4,trump,1101
...,...,...
36541,workingincare,1
36542,healthcareassistant,1
36543,bloodandsilk,1
36544,buy,1


In [8]:
hastags_values.to_csv('data/hashtags.csv', index=False)

## Tweet

In [9]:
text = tweet[['date','text']]

In [10]:
text.insert(2,"lenth",text.text.str.len())

In [11]:
text

,date,text,lenth
0,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,140
1,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,140
2,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,140
3,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,139
4,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,138
...,...,...,...
179103,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,128
179104,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,65
179105,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,140
179106,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",106


In [16]:
mentions = text['text'].str.findall(r'(@\w+)')

In [17]:
text.insert(3,'nbr_mentions', mentions.apply(lambda x : len(x)))

In [18]:
mentions = mentions.explode().value_counts(True)
mentions

text
@realDonaldTrump    0.045632
@WHO                0.007982
@GOP                0.005290
@JoeBiden           0.004834
@BorisJohnson       0.004611
                      ...   
@NITJamshedpur      0.000012
@jamshedpur_nit     0.000012
@Vegalteno          0.000012
@Devendra_1925      0.000012
@Official_SABC1     0.000012
Name: proportion, Length: 38186, dtype: float64

In [19]:
mentions.to_csv('data/mentions.csv')

In [20]:
text

,date,text,lenth,nbr_mentions
0,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,140,0
1,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,140,3
2,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,140,3
3,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,139,1
4,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,138,4
...,...,...,...,...
179103,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,128,4
179104,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,65,0
179105,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,140,1
179106,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...",106,0


In [21]:
def remove_usernames_links(tweet):
    tweet = re.sub('#[^\s]+', '', tweet)
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet) 
    tweet = re.sub(' +', ' ', tweet)
    return tweet


In [22]:
text.insert(2,"text_without_#&@",text.text.apply(remove_usernames_links))

In [23]:
text

,date,text,text_without_#&@,lenth,nbr_mentions
0,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,If I smelled the scent of hand sanitizers toda...,140,0
1,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,Hey and - wouldn't it have made more sense to ...,140,3
2,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,Trump never once claimed was a hoax. We all c...,140,3
3,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,The one gift has give me is an appreciation f...,139,1
4,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,25 July : Media Bulletin on Novel \n,138,4
...,...,...,...,...,...
179103,2020-08-29 19:44:21,Thanks @IamOhmai for nominating me for the @WH...,Thanks for nominating me for the challenge.\n\...,128,4
179104,2020-08-29 19:44:16,2020! The year of insanity! Lol! #COVID19 http...,2020! The year of insanity! Lol!,65,0
179105,2020-08-29 19:44:15,@CTVNews A powerful painting by Juan Lucena. I...,A powerful painting by Juan Lucena. It's a tr...,140,1
179106,2020-08-29 19:44:14,"More than 1,200 students test positive for #CO...","More than 1,200 students test positive for at ...",106,0
